# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

* トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
* 空文字列となったトークンは削除


以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [3]:
import bz2

with bz2.open('enwiki-20150112-400-r10-105752.txt.bz2', 'rt') as f, open('words.txt', 'w') as of:
    for line in f:
        words = line.rstrip().split(' ')
        words = [word.strip('.,!?;:()[]\'"') for word in words]
        output_line = ' '.join([word for word in words if len(word) > 0])
        of.write(output_line + '\n')

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [211]:
import pandas as pd
# https://developers.google.com/public-data/docs/canonical/countries_csv
country_names = [name.rstrip() for name in open ('countries.txt').readlines()]

df = pd.DataFrame(country_names, columns=['Name'])
df = df.sort_values(by='Name')
df['FirstName'] = [name.split(' ')[0] for name in df['Name'].values]
df['WordCount'] = [len(name.split(' ')) for name in df['Name'].values]
df = df[df['WordCount'] > 1]
df.tail()

,Name,FirstName,WordCount
75,United Kingdom,United,2
229,United States,United,2
232,Vatican City,Vatican,2
239,Wallis and Futuna,Wallis,3
63,Western Sahara,Western,2


In [386]:
import re

dic = {}
for i in range(0, df.shape[0]):
    key = df.iloc[i, 1]
    val = list(df.iloc[i, [0, 2]])
    val = df.iloc[i, 0]
    if key not in dic:
        dic[key] = []
    dic[key].append(val)

for k, v in dic.items():
    dic[k] = sorted(v, key=lambda x: len(x), reverse=True)

with open('words.txt') as f, open('words2.txt', 'w') as f2:
    for line in f:
        line = line.rstrip()
        
        for first_name, names in dic.items():
            if re.search(first_name, line) is not None:
                for name in names:
                    if re.search(name, line) is not None:
                        joined_name = name.replace(' ', '_')
                        line = line.replace(name, joined_name)
        f2.write(line + '\n')